# Tarea calificada 1 INAR 2021-22 - Problem Solving

## Grupo INSO3A

A continuación se plantean, dentro de este notebook jupyter, **2 problemas** cuya solución se ha de codificar mediante python (python3) utilizando alguna de las librerías mostradas en clase, **o alguna alternativa si está suficientemente bien documentada**.


# REQUISITOS GENERALES

- El código debe ser **reproducible**, esto es, el profesor o **cualquiera con un entorno python similar al usado en el curso** debe poder ejecutar y reproducir el código **sin error y con resultados apropiados para la formulación del problema**

- El código se entregará en un **notebook de jupyter** independientemente de que el código python haya sido desarrollado en el IDE de elección de cada uno, pero se incluirá en este notebook. **Solo se entregará el notebook, no ficheros .py aparte**

- El fichero que contiene el notebook se llamará  apellido_nombre_INAR2021_tarea1.ipynb

- Este trabajo es **individual**. Dicho esto, se considerarán **nulos los trabajos presentados que sean idénticos o similares más allá de una similaridad razonable en el código utilizado** (todos los que sean similares).

# CALIFICACIÓN

- Cada problema incluido en esta tarea suma 5 puntos de un total de 10, aunque con diferencias en la asignación de esos 5 puntos -leer detenidamente la descripción de cada problema

# Problema 1: Crea un "cryptarithmetic" con la palabra BOMBA

## Descripción del problema

Crea y **resuelve** un problema cryptarithmetic uno de cuyos elementos o palabras será BOMBA. El problema planteado debe tener solución -no vale resultado "INFEASIBLE"- y se debe mostrar la(s) solucion(es) que se encuentren con el código aportado. 

Se pueden plantear problemas con más de dos sumandos, y se valorará la longitud y complejidad de la solución encontrada. Las palabras aportadas deben tener algún significado en español o inglés -no se considerará solución aquella que incluya palabras "aleatorias"-.

## Calificación

- 4 puntos para la solución del problema de acuerdo con la especificación del problema
- 1 punto adicional se reservará "modo concurso", esto es, las soluciones más largas, complejas, o incluso "originales" (a criterio estrictamente subjetivo del profesor) recibirán ese punto adicional o parte del mismo.


In [1]:
"""
En este ejercicio pues lo que hare sera resolver el ejercicio de Cryptarithm donde 
la frase que aplicaré será COMO + PARAR + LA = BOMBA ya que la frase tenia que tener sentido y ademas 
tener la palabra BOMBA. La decision de cojer esta frase fue porque para empezar,
al tratarse de una aplabra en español pues sería mas apropiado usar el español y ademas tiene 
menos de 10 caracteres distintos
"""

from ortools.sat.python import cp_model


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):


    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count


def ComoPararLaBomba():
    
    model = cp_model.CpModel()

    base = 10

    c = model.NewIntVar(1, base - 1, 'C')
    o = model.NewIntVar(0, base - 1, 'O')
    m = model.NewIntVar(0, base - 1, 'M')
    p = model.NewIntVar(1, base - 1, 'P')
    a = model.NewIntVar(0, base - 1, 'A')
    r = model.NewIntVar(0, base - 1, 'R')
    l = model.NewIntVar(1, base - 1, 'L')
    b = model.NewIntVar(1, base - 1, 'B')

    letters = [c, o, m, p, a, r, l,b,]

    assert base >= len(letters)

    model.AddAllDifferent(letters)

    # COMO + PARAR + LA = BOMBA
    model.Add(c * base * base * base + o * base * base + m * base + o + p * base * base * base * base 
              + a * base * base * base + r * base * base + a * base + r + l * base + a 
              == b * base * base * base * base + o * base * base * base + 
              m * base * base + b * base + a)


    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter(letters)

    solver.parameters.enumerate_all_solutions = True

    status = solver.Solve(model, solution_printer)

    print()
    print('Statistics')
    print('  - status          : %s' % solver.StatusName(status))
    print('  - conflicts       : %i' % solver.NumConflicts())
    print('  - branches        : %i' % solver.NumBranches())
    print('  - wall time       : %f s' % solver.WallTime())
    print('  - solutions found : %i' % solution_printer.solution_count())


if __name__ == '__main__':
    ComoPararLaBomba()

C=7 O=1 M=0 P=5 A=3 R=9 L=2 B=6 
C=7 O=2 M=0 P=5 A=4 R=8 L=1 B=6 
C=6 O=1 M=0 P=7 A=4 R=9 L=3 B=8 
C=6 O=2 M=1 P=3 A=5 R=8 L=7 B=4 
C=8 O=1 M=0 P=5 A=2 R=9 L=3 B=6 
C=8 O=1 M=0 P=6 A=2 R=9 L=4 B=7 
C=8 O=3 M=0 P=5 A=4 R=7 L=1 B=6 

Statistics
  - status          : OPTIMAL
  - conflicts       : 566
  - branches        : 916
  - wall time       : 0.037473 s
  - solutions found : 7


# Problema 2: Asignación de turnos (clases) a profesores UTAD

## Descripción del problema

### Problema simple: asignación sin preferencias

Se trata de crear una asignación de los siguientes profesores:

profesores = [ "Mar", "Ramona", "Teresa", "Manoel", "Pedro" ]

a turnos a lo largo de la semana laborable, **solo de lunes a jueves**, de acuerdo a los siguientes **requisitos**:

- Cada día tiene dos turnos: mañana y tarde.
- Cada día cada uno de los dos turnos será asignado **al menos a un profesor** (aclaración: un turno puede albergar varios profesores)
- Ningún profesor trabaja en un mismo día por la mañana **Y** por la tarde.
- Cada profesor será asignado a al menos tres turnos durante el periodo de cuatro días.

### Problema menos simple: asignación con preferencias

Los siguientes profesores tienen preferencia por un turno concreto:

- Ramona: cualquier turno **menos el primer día (lunes)**
- Manoel: turno de mañana en cualquier día
- Pedro: turno de tarde en cualquier día

Adapta la solución del problema para que se cumplan estas preferencias

## Calificación

- 2 puntos para la solución del problema simple
- 2 puntos adicionales para la solución del problema con preferencias (**supone haber solucionado el problema simple**)
- 1 punto adicional se reservará para la calidad de los comentarios y en general la descripción de la solución.


In [2]:
""" Como hay dos ejercicios de estos, pues esta respuesta es para la primera parte, es decir 
    sin preferencias donde lo que tiene que cumplir es que los 5 profesores sean repartidos en 
    los primero 4 dias de la semana con 2 turnos cada dia. Como minimo habra un profesor en 
    cada turno, uno en el de mañana y otro en el de la tarde y en ningun momento habran 
    profesores los cuales tengan que estar en los dos turnos a la vez. Por ultimo cada profesor 
    ha de haber trabajado como minimo 3 veces (o mejor dicho, asistito a) 3 turnos a lo largo
    de la semana.
"""
from ortools.sat.python import cp_model


def main():
    # creo las variables con los numero maximos de profesores, turnos y dias.
    profesoresNum = 5
    turnosNum = 2
    diasNum = 4
    
    #creo los arrays con sus tamaños adecuados
    profesoresTotal = range(profesoresNum)
    turnosTotal = range(turnosNum)
    diasTotal = range(diasNum)
    
    #lso convierto en diccionarios para luego poder usarlos en los print() para poder
    #ver mejor los resultados
    profesoresTotal = {0:"Mar", 1:"Ramona", 2:"Teresa", 3:"Manoel", 4:"Pedro"}
    turnosTotal = {0:"mañana", 1:"tarde"}
    diasTotal = {0: "Lunes",1: "Martes",2: "Miercoles",3: "Jueves"}
    
    model = cp_model.CpModel()

    #Creamos las 3 variables que iran dentro de shift (que es turno) y esta estara compuesto por
    #p de profesor, d de dia, y t de turno
    shifts = {}
    for p in profesoresTotal:
        for d in diasTotal:
            for t in turnosTotal:
                shifts[(p, d,
                        t)] = model.NewBoolVar('shift_n%id%is%i' % (p, d, t))


    # Cada turno le es asignado a 3 profesores como minimo
    for d in diasTotal:
        for t in turnosTotal:
            model.Add(sum(shifts[(p, d, t)] for p in profesoresTotal) < 3)

    # Each nurse works at most one shift per day.
    # cada profesor como mucho trabaja en un turno (el de mañana o el de tarde)
    for p in profesoresTotal:
        for d in diasTotal:
            model.Add(sum(shifts[(p, d, t)] for t in turnosTotal) <= 1)

    #se asignana el minimo de turno que tiene que realizar los profesores a lo largo de toda una semana
    #pero de ser que no se puede completar, un profesor o mas tendran que hacer turnos extra
    minTurnoProfesores = 3
    if turnosNum * diasNum % profesoresNum == 0:
        maxTrunoProfesores = minTurnoProfesores
    else:
        maxTrunoProfesores = minTurnoProfesores + 1
    for p in profesoresTotal:
        turnosNum_worked = []
        for d in diasTotal:
            for t in turnosTotal:
                turnosNum_worked.append(shifts[(p, d, t)])
        model.Add(minTurnoProfesores <= sum(turnosNum_worked))
        model.Add(sum(turnosNum_worked) <= maxTrunoProfesores)

    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    
    solver.parameters.enumerate_all_solutions = True


    class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, profesoresNum, diasNum, turnosNum, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._profesoresNum = profesoresNum
            self._diasNum = diasNum
            self._turnosNum = turnosNum
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print('\nSolucion: %i' % self._solution_count)
            for d in range(self._diasNum):
                print(diasTotal[d])
                for p in range(self._profesoresNum):
                    is_working = False
                    for t in range(self._turnosNum):
                        if self.Value(self._shifts[(p, d, t)]):
                            is_working = True
                            print('  Profesor '+profesoresTotal[p]+' trabaja en la jornada de '+ turnosTotal[t])
                    if not is_working:
                        print('  Profesor '+profesoresTotal[p]+ ' no trabaja')
            if self._solution_count >= self._solution_limit:
                print('Stop search after %i solutions' % self._solution_limit)
                self.StopSearch()

        def solution_count(self):
            return self._solution_count

    solution_limit = 5
    solution_printer = NursesPartialSolutionPrinter(shifts, profesoresNum,
                                                    diasNum, turnosNum,
                                                    solution_limit)

    solver.Solve(model, solution_printer)

    print('\nStatistics')
    print('  - conflicts      : %i' % solver.NumConflicts())
    print('  - branches       : %i' % solver.NumBranches())
    print('  - wall time      : %f s' % solver.WallTime())
    print('  - solutions found: %i' % solution_printer.solution_count())


if __name__ == '__main__':
    main()


Solucion: 1
Lunes
  Profesor Mar no trabaja
  Profesor Ramona trabaja en la jornada de mañana
  Profesor Teresa trabaja en la jornada de mañana
  Profesor Manoel trabaja en la jornada de tarde
  Profesor Pedro trabaja en la jornada de tarde
Martes
  Profesor Mar trabaja en la jornada de mañana
  Profesor Ramona no trabaja
  Profesor Teresa trabaja en la jornada de mañana
  Profesor Manoel trabaja en la jornada de tarde
  Profesor Pedro trabaja en la jornada de tarde
Miercoles
  Profesor Mar trabaja en la jornada de tarde
  Profesor Ramona trabaja en la jornada de tarde
  Profesor Teresa no trabaja
  Profesor Manoel trabaja en la jornada de mañana
  Profesor Pedro trabaja en la jornada de mañana
Jueves
  Profesor Mar trabaja en la jornada de tarde
  Profesor Ramona trabaja en la jornada de mañana
  Profesor Teresa trabaja en la jornada de mañana
  Profesor Manoel no trabaja
  Profesor Pedro no trabaja

Solucion: 2
Lunes
  Profesor Mar no trabaja
  Profesor Ramona trabaja en la jornada 

In [3]:
"""En este caso pues se trata con la asigancion con preferencia donde solo 3 de estos profesores
    tienen preferencias que son Ramona, la cual no quiere trabajar los lunes, Manoel quien quiere trabajar 
    solo por las mañanas y por ultimo Pedro quien solo quiere trabajar por las tardes. Tambien
    una cosita mas y es que como los dos codigos son similares, pues no he puestos algunos de los
    comentarios como antes
"""
from ortools.sat.python import cp_model


def main():
    # El mismo set de informacion que en el de sin preferencias.
    profesoresNum = 5
    turnosNum = 2
    diasNum = 4
    
    profesoresTotal = range(profesoresNum)
    turnosTotal = range(turnosNum)
    diasTotal = range(diasNum)
    
    profesoresTotal = {0:"Mar", 1:"Ramona", 2:"Teresa", 3:"Manoel", 4:"Pedro"}
    turnosTotal = {0:"mañana", 1:"tarde"}
    diasTotal = {0: "Lunes",1: "Martes",2: "Miercoles",3: "Jueves"}
    
    #Aqui pues tenemos  profesores que son 5, repartidos cada uno en 4 dias y cada dias con 2 shift
    turnoConPreferencia = [[[0, 0], [0, 0], [0, 0], [0, 0]], #MAR
                      [[0, 0], [1, 1], [1, 1], [1, 1]], #RAMONA: quiere trabajar cualquier dia que no se el Lunes
                      [[0, 0], [0, 0], [0, 0], [0, 0]], #TERESA
                      [[1, 0], [1, 0], [1, 0], [1, 0]], #MANOEL: quiere trabajar cualquier dia por la mañana
                      [[0, 1], [0, 1], [0, 1], [0, 1]]] #PEDRO: quiere trabajar cualquier dia por la tarde
                      #LUNES  MARTES  MIERCOLES JUEVES

    model = cp_model.CpModel()

    shifts = {}
    for p in profesoresTotal:
        for d in diasTotal:
            for t in turnosTotal:
                shifts[(p, d,
                        t)] = model.NewBoolVar('shift_n%id%is%i' % (p, d, t))

  
    for d in diasTotal:
        for t in turnosTotal:
            model.Add(sum(shifts[(p, d, t)] for p in profesoresTotal) < 3)


    for p in profesoresTotal:
        for d in diasTotal:
            model.Add(sum(shifts[(p, d, t)] for t in turnosTotal) <= 1)
            
    minTurnoProfesores = 3
    if turnosNum * diasNum % profesoresNum == 0:
        maxTrunoProfesores = minTurnoProfesores
    else:
        maxTrunoProfesores = minTurnoProfesores + 1
    for p in profesoresTotal:
        turnosNum_worked = 0
        for d in diasTotal:
            for t in turnosTotal:
                turnosNum_worked += shifts[(p, d, t)]
        model.Add(minTurnoProfesores <= turnosNum_worked)
        model.Add(turnosNum_worked <= maxTrunoProfesores)

        
    model.Maximize(
        sum(turnoConPreferencia[p][d][t] * shifts[(p, d, t)] for p in profesoresTotal
            for d in diasTotal for t in turnosTotal))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        print('Solucion:')
        for d in diasTotal:
            print(diasTotal[d])
            for p in profesoresTotal:
                for t in turnosTotal:
                    if solver.Value(shifts[(p, d, t)]) == 1:
                        if turnoConPreferencia[p][d][t] == 1:
                            print('Profesor ', profesoresTotal[p], ' trabaja en la jornada de ', turnosTotal[t],
                                  '(pedido).')
                        else:
                            print('Profesor ', profesoresTotal[p], ' trabaja en la jornada de ', turnosTotal[t],
                                  '(no pedido).')
            print()
        print(f'Number of shift requests met = {solver.ObjectiveValue()}',
              f'(out of {profesoresNum * minTurnoProfesores})')
    else:
        print('No optimal solution found !')


    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())


if __name__ == '__main__':
    main()

Solucion:
Lunes
Profesor  Mar  trabaja en la jornada de  tarde (no pedido).
Profesor  Teresa  trabaja en la jornada de  mañana (no pedido).
Profesor  Manoel  trabaja en la jornada de  mañana (pedido).
Profesor  Pedro  trabaja en la jornada de  tarde (pedido).

Martes
Profesor  Mar  trabaja en la jornada de  mañana (no pedido).
Profesor  Ramona  trabaja en la jornada de  tarde (pedido).
Profesor  Manoel  trabaja en la jornada de  mañana (pedido).
Profesor  Pedro  trabaja en la jornada de  tarde (pedido).

Miercoles
Profesor  Ramona  trabaja en la jornada de  mañana (pedido).
Profesor  Teresa  trabaja en la jornada de  tarde (no pedido).
Profesor  Manoel  trabaja en la jornada de  mañana (pedido).
Profesor  Pedro  trabaja en la jornada de  tarde (pedido).

Jueves
Profesor  Mar  trabaja en la jornada de  mañana (no pedido).
Profesor  Ramona  trabaja en la jornada de  mañana (pedido).
Profesor  Teresa  trabaja en la jornada de  tarde (no pedido).
Profesor  Pedro  trabaja en la jornada de  